In [1]:
from pathlib import Path

import numpy as np
import yaml
import tensorflow.compat.v1 as tf

# from contact_graspnet.orig.contact_graspnet.data import load_available_input_data
from contact_graspnet.orig.contact_graspnet.visualization_utils import visualize_grasps, show_image

from contact_graspnet.models import ContactGraspnet
from contact_graspnet.dataloading import YCBSimulationData

2023-02-18 17:00:25.418703: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-18 17:00:26.546680: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/moritz/catkin_ws/devel/lib:/opt/ros/noetic/lib:/home/moritz/.mujoco/mujoco210/bin:/home/moritz/mambaforge/envs/contact/lib/:/home/moritz/mambaforge/envs/contact/lib/
2023-02-18 17:00:26.546804: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No

/home/moritz/Documents/ContactGraspnetBenchmark/contact_graspnet/orig/pointnet2/tf_ops/sampling


In [ ]:

tf.disable_eager_execution()
physical_devices = tf.config.experimental.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

2023-02-18 16:56:51.577250: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-18 16:56:51.586776: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-18 16:56:51.587201: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [ ]:
checkpoint_dir = Path.cwd().parent / "contact_graspnet" / "checkpoints" / "scene_test_2048_bs3_hor_sigma_001"
config_path = checkpoint_dir / "config.yaml"

example_data_path = Path.cwd().parent / "contact_graspnet" / "data" / "test_data" / "7.npy"
ycb_data_path = Path.home() / "Documents" / "ycb_sim_data_1"

In [ ]:
ycb_sim_datset = YCBSimulationData(ycb_data_path)
ycb_sample = ycb_sim_datset[0]

In [ ]:
with open(config_path,'r') as f:
    global_config = yaml.safe_load(f)

global_config['OPTIMIZER']['batch_size'] = 1    


In [ ]:
model = ContactGraspnet(global_config, checkpoint_dir)

--- Get model
Instructions for updating:
Colocations handled automatically by placer.


/home/moritz/Documents/ContactGraspnetBenchmark/contact_graspnet/orig/pointnet2/utils/tf_util.py:571: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  momentum=bn_decay, axis=axis, name=scope,


KeyboardInterrupt: 

In [ ]:
# points = sample.points.astype(np.float32)

# model(points)
# segmap, rgb, depth, cam_K, _, _ = load_available_input_data(str(example_data_path))
# pc_full, pc_segments, pc_colors = extract_point_clouds(
#     depth,
#     cam_K,
#     segmap=segmap,
#     rgb=rgb,
#     skip_border_objects=False,
#     z_range=(0.2, 1.8),
# )

model(sample.points)


Generated 12 grasps


({-1: array([[[-0.23826635,  0.8339543 ,  0.49774417, -0.12402885],
          [ 0.06275294, -0.4982137 ,  0.86478037, -0.10142914],
          [ 0.96917033,  0.23728296,  0.06637449,  0.87399155],
          [ 0.        ,  0.        ,  0.        ,  1.        ]],
  
         [[ 0.22374351,  0.69013715, -0.6882221 ,  0.0966803 ],
          [-0.03384801,  0.71119976,  0.70217466, -0.05295241],
          [ 0.9740601 , -0.13381208,  0.18248612,  0.86197317],
          [ 0.        ,  0.        ,  0.        ,  1.        ]],
  
         [[ 0.10822622,  0.9940507 ,  0.01224815, -0.00731822],
          [-0.25986555,  0.01639616,  0.96550554, -0.14887668],
          [ 0.95956075, -0.1076759 ,  0.26009405,  0.85367626],
          [ 0.        ,  0.        ,  0.        ,  1.        ]],
  
         [[ 0.06767599,  0.9976984 , -0.0042058 , -0.04800991],
          [-0.19490418,  0.01735477,  0.98066866, -0.14577717],
          [ 0.9784847 , -0.065548  ,  0.1956301 ,  0.86060077],
          [ 0.        , 